In [1]:
import mne
import numpy as np
import os
from glob import glob

from mne_bids import BIDSPath, write_raw_bids

In [8]:
'''# project directory
proj_dir = os.path.join('/Users/dsj3886/',
                        'data_local',
                        #'EAM1',
                        'EAM1_local',
                        )

# raw data directory (need to specify for your computer)
data_dir = os.path.join(proj_dir, 'EEG_raw')
'''
# on Mac Studio Sharepoint
proj_dir = os.path.join('/Users/dsj3886/',
                         'Library/CloudStorage/',
                         'OneDrive-NorthwesternUniversity',
                         'Documents - SoundBrain Lab',
                         'Lab Research Projects/',
                         'Experiments/NU_Experiments/',
                         'EAM1/')
data_dir = os.path.join(proj_dir, 'EEG_raw')

In [5]:
# output BIDS directory to be created/filled
bids_dir = os.path.join(proj_dir,
                        'data-bids')

In [33]:
# define function to generate event labels and codes
# (differs per participant)
# TODO: add motor condition
# TODO: fix sub < 13 task-active codes
def generate_event_dict(sub_label, task_label):
    if int(sub_label) > 19:
        event_dict = {'passive/pos': 1,
                    'passive/neg': 2,
                    'active/pos': 3,
                    'active/neg': 4}
    elif int(sub_label) > 13:
        event_dict = {f'{task_label}/pos': 1,
                    f'{task_label}/neg': 2,}
    elif int(sub_label) < 13:
        if task_label == 'passive':
            event_dict = {f'{task_label}/pos': 2049,
                        f'{task_label}/neg': 2050,}
        elif task_label == 'active':
            print('all active coded as 2049.',
                  '  need to fix')
            event_dict = {f'{task_label}/pos': 2049,
                        f'{task_label}/neg': 2050,}
            #pass
    return event_dict

In [ ]:
# define input EEG file
# bad participants: 01, 08, 13
for sub_num in range(2, 35):
    for task_label in ['active', 'passive']:
        for run_label in range(1,6):
            sub_label = f'{sub_num:02d}'

            data_fpath = os.path.join(data_dir,
                                    f'sub-{sub_label}_task-{task_label}_run-{run_label}.bdf')
            
            try:
                # read in input EEG file
                raw_data = mne.io.read_raw_bdf(data_fpath)

                # find events in the `Status` channel of the raw data
                events_raw = mne.find_events(raw_data, 
                                            stim_channel='Status', 
                                            initial_event=True)
                '''
                # explore the data
                events_raw[events_raw[:,2]==2049]
                '''
                '''
                # Modify event codes based on run type
                if int(sub_label) < 13:
                    if task_label=='active':
                        # change odd-indexed sound events to code 2050
                        odd_events = [] # TODO - array of odd indices
                        events_raw[events_raw[:,2]==2049][odd_events] = 2050
                '''        
                # run the function to generate events
                event_dict = generate_event_dict(sub_label, task_label)
                print('task events:', event_dict)

                # only keep task-related events (currently active and passive runs only)
                events = mne.pick_events(events_raw, include=[1, 2, 3, 4, 2049, 2050])
                print('task event codes in this run:', np.unique(events[:,2]))

                # define the BIDSPath object for converting this EEG file
                bids_path = BIDSPath(subject=str(sub_label), 
                                    run=run_label, 
                                    task=task_label,
                                    datatype='eeg', 
                                    root=bids_dir, )
                
                # finally, do the raw-to-BIDS conversion
                write_raw_bids(raw_data, 
                            bids_path=bids_path,   
                            events=events,
                            event_id=event_dict,
                            overwrite=True
                            )
            except FileNotFoundError:
                print('file does not exist:', data_fpath)
            except ValueError:
                print('need to fix events issue')

Extracting EDF parameters from /Users/dsj3886/Library/CloudStorage/OneDrive-NorthwesternUniversity/Documents - SoundBrain Lab/Lab Research Projects/Experiments/NU_Experiments/EAM1/EEG_raw/sub-09_task-active_run-1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
4843 events found on stim channel Status
Event IDs: [ 2048  2049  2056  2177  2303 34816 34817 34818 34824 34945 34952 67584]
all active coded as 2049.   need to fix
task events: {'active/pos': 2049, 'active/neg': 2050}
task event codes in this run: [2049]
Extracting EDF parameters from /Users/dsj3886/Library/CloudStorage/OneDrive-NorthwesternUniversity/Documents - SoundBrain Lab/Lab Research Projects/Experiments/NU_Experiments/EAM1/EEG_raw/sub-09_task-active_run-1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Writing '/Users/dsj3886/Library/CloudStorage/OneDrive-NorthwesternUniversity/Documents - SoundBrain Lab/Lab Research Projects/Experiments

In [39]:
(f'{sub_num:02d}')

'08'